<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C11-Offensive_Forensics/User_Reconnaissance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__User Reconnaissance__*

Now let's do some recon on the user of the VM. The __cmdline__ plug-in lists the command line arguments for each process as they were running at the time the snapshot was made. These processes give us a hint to the user's behavior and intent.

```
PS> vol -f WinDev2007Eval-7d959ee5.vmem windows.cmdline
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
PID Process Args
72 Registry Required memory at 0x20 is not valid (process exited?)
340 smss.exe Required memory at 0xa5f1873020 is inaccessible (swapped)
564 lsass.exe C:\Windows\system32\lsass.exe
624 winlogon.exe winlogon.exe
2160 MsMpEng.exe "C:\ProgramData\Microsoft\Windows Defender\platform\4.18.2008.9-0\
MsMpEng.exe"
4732 explorer.exe C:\Windows\Explorer.EXE
4848 svchost.exe C:\Windows\system32\svchost.exe -k ClipboardSvcGroup -p
4920 dllhost.exe C:\Windows\system32\DllHost.exe /Processid:{AB8902B4-09CA-4BB6-B78DA8F59079A8D5}
5084 StartMenuExper "C:\Windows\SystemApps\Microsoft.Windows. . ."
5388 MicrosoftEdge. "C:\Windows\SystemApps\Microsoft.MicrosoftEdge_. . ."
6452 OneDrive.exe "C:\Users\Administrator\AppData\Local\Microsoft\OneDrive\OneDrive.exe"
/background
6484 FreeDesktopClo "C:\Program Files\Free Desktop Clock\FreeDesktopClock.exe"
7092 cmd.exe "C:\Windows\system32\cmd.exe" #[1]
3312 notepad.exe notepad #[2]
3824 powershell.exe "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"
6448 Calculator.exe "C:\Program Files\WindowsApps\Microsoft.WindowsCalculator_. . ."
6684 firefox.exe "C:\Program Files (x86)\Mozilla Firefox\firefox.exe"
6432 PowerToys.exe "C:\Program Files\PowerToys\PowerToys.exe"
7124 nc64.exe Required memory at 0x2d7020 is inaccessible (swapped)
3324 smartscreen.ex C:\Windows\System32\smartscreen.exe -Embedding
4768 ipconfig.exe Required memory at 0x840308e020 is not valid (process exited?)
```

The list shows the process ID, process name, and command line with arguments that started the process. You can see that most processes were started by the system itself, most likely at boot time. The __cmd.exe__ __[1]__ and __notepad.exe__ __[2]__ processes are typical processes a user would start.

Let's investigate the running processes a little bit deeper with the __pslist__ plug-in, which lists the proceses that were running at the time of the snapshot.

```
PS> vol -f WinDev2007Eval-7d959ee5.vmem windows.pslist
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
PID PPID ImageFileName Offset(V) Threads Handles SessionId Wow64
4 0 System 0xa50bb3e6d040 129 - N/A False
72 4 Registry 0xa50bb3fbd080 4 - N/A False
6452 4732 OneDrive.exe 0xa50bb4d62080 25 - 1 True
6484 4732 FreeDesktopClo 0xa50bbb847300 1 - 1 False
6212 556 SgrmBroker.exe 0xa50bbb832080 6 - 0 False
1636 556 svchost.exe 0xa50bbadbe340 8 - 0 False
7092 4732 cmd.exe 0xa50bbbc4d080 1 - 1 False
3312 7092 notepad.exe 0xa50bbb69a080 3 - 1 False
3824 4732 powershell.exe 0xa50bbb92d080 11 - 1 False
6448 704 Calculator.exe 0xa50bb4d0d0c0 21 - 1 False
4036 6684 firefox.exe 0xa50bbb178080 0 - 1 True
6432 4732 PowerToys.exe 0xa50bb4d5a2c0 14 - 1 False
4052 4700 PowerLauncher. 0xa50bb7fd3080 16 - 1 False
5340 6432 Microsoft.Powe 0xa50bb736f080 15 - 1 False
8564 4732 python-3.8.6-a 0xa50bb7bc2080 1 - 1 True
7124 7092 nc64.exe 0xa50bbab89080 1 - 1 False
3324 704 smartscreen.ex 0xa50bb4d6a080 7 - 1 False
7364 4732 cmd.exe 0xa50bbd8a8080 1 - 1 False
8916 2136 cmd.exe 0xa50bb78d9080 0 - 0 False
4768 8916 ipconfig.exe 0xa50bba7bd080 0 - 0 False
```

Here we see the actual processes and their memory offsets. Some columns have been omitted for space. Several interesting processes are listed, including the __cmd__ and __notepad__ processes we saw in the output from the __cmdline__ plug-in.

It would be nice to see the processes as a hierarchy, so we can tell what process started other processes. For that, we'll use the __pstree__ plug-in:

```
PS> vol -f WinDev2007Eval-7d959ee5.vmem windows.pstree
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
PID PPID ImageFileName Offset(V) Threads Handles SessionId Wow64
4 0 System 0xa50bba7bd080 129 N/A False
* 556 492 services.exe 0xa50bba7bd080 8 0 False
** 2176 556 wlms.exe 0xa50bba7bd080 2 0 False
** 1796 556 svchost.exe 0xa50bba7bd080 13 0 False
** 776 556 svchost.exe 0xa50bba7bd080 15 0 False
** 8 556 svchost.exe 0xa50bba7bd080 18 0 False
*** 4556 8 ctfmon.exe 0xa50bba7bd080 10 1 False
*** 5388 704 MicrosoftEdge. 0xa50bba7bd080 35 1 False
*** 6448 704 Calculator.exe 0xa50bba7bd080 21 1 False
*** 3324 704 smartscreen.ex 0xa50bba7bd080 7 1 False
** 2136 556 vmtoolsd.exe 0xa50bba7bd080 11 0 False
*** 8916 2136 cmd.exe 0xa50bba7bd080 0 0 False
**** 4768 8916 ipconfig.exe 0xa50bba7bd080 0 0 False
Offensive Forensics 175
* 4704 624 userinit.exe 0xa50bba7bd080 0 1 False
** 4732 4704 explorer.exe 0xa50bba7bd080 92 1 False
*** 6432 4732 PowerToys.exe 0xa50bba7bd080 14 1 False
**** 5340 6432 Microsoft.Powe 0xa50bba7bd080 15 1 False
*** 7364 4732 cmd.exe 0xa50bba7bd080 1 - False
**** 2464 7364 conhost.exe 0xa50bba7bd080 4 1 False
*** 7092 4732 cmd.exe 0xa50bba7bd080 1 - False
**** 3312 7092 notepad.exe 0xa50bba7bd080 3 1 False
**** 7124 7092 nc64.exe 0xa50bba7bd080 1 1 False
*** 8564 4732 python-3.8.6-a 0xa50bba7bd080 1 1 True
**** 1036 8564 python-3.8.6-a 0xa50bba7bd080 5 1 True
```

Now we get a clearer picture. The asterisk in each row indicates the parent-child relationship of the process. For example, the __userint__ process (__PID 4704__) spawned the __explorer.exe__ process (__PID 4732__) started the __cmd.exe__ process (__PID 7092__). From that process, the user started __notepad.exe__ and another process called __nc64.exe__.

Now let's check for passwords with the __hashdump__ plug-in:

```
PS> vol -f WinDev2007Eval-7d959ee5.vmem windows.hashdump
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
User rid lmhash nthash
Administrator 500 aad3bXXXXXXaad3bXXXXXX fc6eb57eXXXXXXXXXXX657878
Guest 501 aad3bXXXXXXaad3bXXXXXX 1d6cfe0dXXXXXXXXXXXc089c0
DefaultAccount 503 aad3bXXXXXXaad3bXXXXXX 1d6cfe0dXXXXXXXXXXXc089c0
WDAGUtilityAccount 504 aad3bXXXXXXaad3bXXXXXX ed66436aXXXXXXXXXXX1bb50f
User 1001 aad3bXXXXXXaad3bXXXXXX 31d6cfe0XXXXXXXXXXXc089c0
tim 1002 aad3bXXXXXXaad3bXXXXXX afc6eb57XXXXXXXXXXX657878
admin 1003 aad3bXXXXXXaad3bXXXXXX afc6eb57XXXXXXXXXXX657878
```

The output shows the account usernames and the LM and NT hashes of their passwords. Recovering the password hashes on a Windows machine after penetration is a common goal of attackers. These hashes can be cracked offline in an attempt to recover the target's password, or they can be used in a pass-the-hash attack to gain access to other network resources. Whether the target is a paranoid user who performs high-risk operations only on a VM or is an enterprise attempting to contain some of its user's activities to VMs, looking through the VMs or snapshots on the system is a perfect time for attempting to recover these hashes after you've gained access to the host hardware.

Volatility makes this recovery process extremely easy.

We've obfuscated the hashes in our output. You can use your own output as input to a hash-cracking tool to find your way into the VM. There are several online hash-cracking sites; alternatively, you can use __John the Ripper__ on your Kali Machine.